In [ ]:
!pip install dedupe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 KB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.7/284.7 KB 14.3 MB/s eta 0:00:00


In [ ]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 16.1 MB/s eta 0:00:00


In [ ]:
import os
import csv
import re
import logging
import optparse

import dedupe
from unidecode import unidecode

In [ ]:
def preProcess(column):

  column = unidecode(column)
  column = re.sub('\n', ' ', column)
  column = re.sub('-', '', column)
  column = re.sub('/', ' ', column)
  column = re.sub("'", '', column)
  column = re.sub(",", '', column)
  column = re.sub(":", ' ', column)
  column = re.sub('  +', ' ', column)
  column = column.strip().strip('"').strip("'").lower().strip()
  if not column:
    column = None
  return column

def readData(filename):

  data_d = {}

  with open(filename, encoding = "ISO-8859-1") as f:
    reader = csv.DictReader(f)
    for i, row in enumerate(reader):
      clean_row = dict([(k, preProcess(v)) for (k, v) in row.items()])
      if clean_row['price']:
        clean_row['price'] = float(clean_row['price'][1:])
      data_d[clean_row['id']] = dict(clean_row)

  return data_d


In [ ]:
def generate_train_test_data(file1):
  abt_train = file1 + "/abt_train.csv"
  abt_test = file1 + "/abt_test.csv"

  buy_train = file1 + "/buy_train.csv"
  buy_test = file1 + "/buy_test.csv"

  data_abt_train = readData(abt_train)
  data_abt_test = readData(abt_test)

  data_buy_train = readData(buy_train)
  data_buy_test = readData(buy_test)

  return data_abt_train, data_abt_test, data_buy_train, data_buy_test

In [ ]:
data_abt_train, data_abt_test, data_buy_train, data_buy_test = generate_train_test_data("/content/drive/MyDrive/dedupe/21-14")

In [ ]:
# left_file = "/content/drive/MyDrive/RLTK/Abt.csv"
# right_file = "/content/drive/MyDrive/RLTK/Buy.csv"

In [ ]:
# print('importing data ...')
# data_1 = readData(left_file)
# data_2 = readData(right_file)

importing data ...


In [ ]:
def descriptions():
  for dataset in (data_abt_train, data_buy_train):
    for record in dataset.values():
      yield record['description']

In [ ]:
fields = [
            {'field': 'name', 'type': 'String'},
            {'field': 'name', 'type': 'Text', 'corpus': descriptions()},
            {'field': 'description', 'type': 'Text',
             'has missing': True, 'corpus': descriptions()},
            {'field': 'price', 'type': 'Price', 'has missing': True}]

In [ ]:
linker = dedupe.RecordLink(fields)

In [ ]:
# settings_file = "/content/drive/MyDrive/dedupe/21-14/settings_file"

# if os.path.exists(settings_file):
#   print('reading from', settings_file)
#   with open(settings_file, 'rb') as sf:
#       linker = dedupe.StaticRecordLink(sf)

reading from /content/drive/MyDrive/dedupe/10-8/settings_file


In [ ]:
len(data_abt_train), len(data_abt_test), len(data_buy_train), len(data_buy_test)

(349, 732, 352, 740)

In [ ]:
import time
start_time = time.time()

with open("/content/drive/MyDrive/dedupe/21-14/training_file") as tf:

  linker.prepare_training(data_abt_train, data_buy_train,
                        training_file=tf)
print("Preparing Training:",(time.time()-start_time))

Preparing Training: 4.858708381652832


In [ ]:
# linker.prepare_training(data_1, data_2, sample_size=15000)

In [ ]:
# dedupe.console_label(linker)

In [ ]:
import time

start_time = time.time()

linker.train()
print("Preparing Training:",(time.time()-start_time))

Preparing Training: 2.2549567222595215


In [ ]:
# with open("/content/drive/MyDrive/dedupe/21-14/training_file", 'w') as tf:
#   linker.write_training(tf)

with open("/content/drive/MyDrive/dedupe/21-14/settings_file", 'wb') as sf:
  linker.write_settings(sf)

In [ ]:
start_time = time.time()

linked_records = linker.join(data_abt_test, data_buy_test, 0.3)

print("Clustering:",(time.time()-start_time))

Clustering: 1.2124402523040771


In [ ]:
len(linked_records)

457

In [ ]:
cluster_membership = {}
for cluster_id, (cluster, score) in enumerate(linked_records):
  for record_id in cluster:
    cluster_membership[record_id] = {'Cluster ID': cluster_id,
                                      'Link Score': score}

In [ ]:
cluster_membership

In [ ]:
len(list(cluster_membership.keys())),len(set(list(cluster_membership.keys())))

(914, 914)

In [ ]:
total_cluster = []
for i,v in cluster_membership.items():
  total_cluster.append(v['Cluster ID'])

In [ ]:
len(total_cluster), len(set(total_cluster))

(914, 457)

In [ ]:
with open("/content/drive/MyDrive/dedupe/21-14/output_file.csv", 'w') as f:

  header_unwritten = True

  for fileno, filename in enumerate(("/content/drive/MyDrive/dedupe/21-14/buy_test.csv", "/content/drive/MyDrive/dedupe/21-14/abt_test.csv")):
    with open(filename, encoding= "ISO-8859-1") as f_input:
      reader = csv.DictReader(f_input)

      if header_unwritten:

        fieldnames = (['Cluster ID', 'Link Score', 'source file'] +
                      reader.fieldnames)

        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

        header_unwritten = False

      for row_id, row in enumerate(reader):

        record_id = str(row['id'])
        cluster_details = cluster_membership.get(record_id, {})
        # print(cluster_details)
        row['source file'] = fileno
        row.update(cluster_details)

        writer.writerow(row)


In [ ]:
import pandas as pd

out = pd.read_csv("/content/drive/MyDrive/dedupe/21-14/output_file.csv")

In [ ]:
out

,Cluster ID,Link Score,source file,id,name,description,manufacturer,price
0,163.0,0.871543,0,10011646,Linksys EtherFast EZXS88W Ethernet Switch - EZ...,Linksys EtherFast 8-Port 10/100 Switch (New/Wo...,LINKSYS,NaN
1,393.0,0.531071,0,10140760,Linksys EtherFast EZXS55W Ethernet Switch,5 x 10/100Base-TX LAN,LINKSYS,NaN
2,446.0,0.337337,0,10221960,Netgear ProSafe FS105 Ethernet Switch - FS105NA,NETGEAR FS105 Prosafe 5 Port 10/100 Desktop Sw...,Netgear,NaN
3,279.0,0.748726,0,10246269,Belkin Pro Series High Integrity VGA/SVGA Moni...,1 x HD-15 - 1 x HD-15 - 10ft - Beige,Belkin,NaN
4,103.0,0.905290,0,10316920,LaCie Pocket Floppy Disk Drive - 706018,LaCie Pocket USB Floppy 1.44 MB,LACIE,NaN
...,...,...,...,...,...,...,...,...
1467,NaN,NaN,1,39039,Garmin Nuvi 885T GPS Navigation System - 01000...,Garmin Nuvi 885T GPS Navigation System - 01000...,NaN,$799.00
1468,439.0,0.360535,1,39054,OmniMount 37' - 63' Ultra Low Profile Series L...,OmniMount 37' - 63' Ultra Low Profile Series L...,NaN,NaN
1469,314.0,0.679097,1,39088,Logitech Cordless Desktop Wave Keyboard And Mo...,Logitech Cordless Desktop Wave Keyboard And Mo...,NaN,$79.00
1470,NaN,NaN,1,39090,Mitsubishi DLP Black TV Stand - MBS73V,Mitsubishi DLP Black TV Stand - MBS73V/ Matchi...,NaN,$549.00


In [ ]:
out.describe()

,Cluster ID,Link Score,source file,id
count,914.000000,914.000000,1472.000000,1.472000e+03
mean,228.000000,0.755235,0.497283,9.795060e+07
std,131.996449,0.178355,0.500163,1.015951e+08
min,0.000000,0.311030,0.000000,4.696000e+03
25%,114.000000,0.642694,0.000000,3.385500e+04
50%,228.000000,0.806076,0.000000,1.028159e+07
75%,342.000000,0.898845,1.000000,2.069205e+08
max,456.000000,0.981909,1.000000,2.106478e+08


In [ ]:
len(out['Cluster ID'].unique()) #including nan

458

In [ ]:
truth_yes = pd.read_csv("/content/drive/MyDrive/dedupe/21-14/test.csv")

In [ ]:
truth_yes['id1']

0       38477
1       38475
2       27248
3       36260
4       35810
        ...  
1453    36124
1454    34724
1455    34724
1456    37192
1457    35900
Name: id1, Length: 1458, dtype: int64

In [ ]:
len(truth_yes)

1458

In [ ]:
truth_yes.head()

,id1,id2,label
0,38477,10011646,True
1,38475,10140760,True
2,27248,10246269,True
3,36260,10316920,True
4,35810,10326220,True


In [ ]:
def truth_eval(out, truth_yes):
  truth_mapping = []
  total_true = 0
  total_false = 0
  false_true = 0
  false_false = 0
  for i, val in truth_yes.iterrows():
    abt_cluster = out[out['id']==val['id1']]['Cluster ID'].item()
    buy_cluster = out[out['id']==val['id2']]['Cluster ID'].item()
    if val['label']==True:
      if abt_cluster==buy_cluster:
        truth_mapping.append(abt_cluster)
        total_true+=1
      else:
        truth_mapping.append(None)
        total_false+=1
    else:
      if abt_cluster==buy_cluster:
        truth_mapping.append(abt_cluster)
        false_true+=1
      else:
        truth_mapping.append(None)
        false_false+=1
  return truth_mapping, total_true, total_false, false_true, false_false

In [ ]:
lst, true_true, true_false, false_true, false_false = truth_eval(out, truth_yes)

In [ ]:
print(true_true, true_false, false_true, false_false)

438 298 8 714


In [ ]:
def metrics(TP,FP,FN):

  recall = TP/(TP+FN)
  precision = TP/(TP+FP)
  fscore = (2*precision*recall)/(precision+recall)

  return recall, precision, fscore

In [ ]:
metrics(true_true, false_true, true_false)

(0.595108695652174, 0.9820627802690582, 0.7411167512690355)

In [ ]:
metrics(628,118,125)

(0.8339973439575034, 0.8418230563002681, 0.8378919279519681)